In [1]:
import sys
sys.path.append("..")

import numpy as np
from matplotlib import pyplot as plt

from amgqcd.linear_solver.cg import *

from amgqcd.multigrid.two_grid import TwoGridQCD, TwoGridBase
from amgqcd.multigrid.three_grid import ThreeGridQCD
from amgqcd.dirac.D_sparse import *

# from amgqcd.dirac.D_sparse_csr import *
# from amgqcd.dirac.D_sparse2 import CalculatorD2 

from load_correct_indices import *
from helper_functions import solve_with_different_precisions

#fixed!!

from scipy.sparse.linalg import LinearOperator

# Loading my code (You do not need to understand this, you can also just ask me)

In [2]:
# Load previously generated configurations

Nx = 128
Nt = 128
lattice_volume = Nt*Nx
Nc = 1
Ns = 2
nDim = 2
gauge_links_array = np.load("configurations/ym_su1_beta6_{}.npy".format(Nt)).reshape(100,Nt*Nx, nDim, Nc,Nc)

# Choose one of the generated gauge field
gauge_links = gauge_links_array[0]


# Choose fermion mass
m = -0.066


# Start Timoteo's Multigrid implementation
mg = TwoGridQCD(gauge_links, Nt, Nx,Nc,Ns)
mg.update_m(m)



# An example of a preconditioner: Multigrid preconditioner (you do not need to understand this deeply)

## The matrix from this example

In [3]:
A = mg.A.copy() # The matrix that needs to be inverted

np.random.seed(0)
b = np.random.randn(Nt*Nx*Ns) + 1j*np.random.randn(Nt*Nx*Ns) # A random vector as the RHS of the linear equation

## Prepare multigrid algorithm (Irrelevant for you)

In [4]:
mg.run_setup_phase("standard", initial_relax = False, numRelax = 300) 
mg.create_operators(mg.B) # This uses the near nullspace to construct the multigrid method

## One interation of the multigrid method by itself

In [5]:
np.random.seed(0)
x = np.random.randn(A.shape[0]).astype(A.dtype)
x0 = x.copy()  # Initial Guess

x = mg.solve(b, x = x)

initial_residue = np.linalg.norm(b - A.dot(x0))  # Measure the quality of the initial guess
residue_after_one_mg = np.linalg.norm(b - A.dot(x)) # Measure how close you are from the correct solution

print(initial_residue)
print(residue_after_one_mg)

1127.7700854908637
176.03995027529442


mg.solve(b) approximates an inversion of A, so it reduces the residue by one order of magnitude. If you use this as a preconditioner for the CG algorithm below, then it can solve the linear equation with considerably fewer iterations

In [6]:
# This allows to use the MG as a preconditioner using solvers from scipy like the conjugate gradient (CG) algorithm
def start_preconditioner(A, mg):
    # Parameters for LinearOperator
    shape = A.shape
    dtype = A.dtype

    def matvec(b):
        # A function which approximately calculates A**{-1}*b
        return mg.solve(b)
    
    # LinearOperator is a class of scipy to make your approximate solver a preconditioner
    return LinearOperator(shape, matvec, dtype=dtype)

M = start_preconditioner(A, mg)

#### Use the MG preconditioner to solve a linear equation with less iterations

In [7]:
x, numIter = cg(A,b, M = M, rtol = 1e-12) # M = M to use the preconditioner defined above
print(np.linalg.norm(A.dot(x) - b)) # Calculate the residual to check if the equation was solved correctly
print(numIter) # If a good near nullspace B was calculated, it should reduce the number of iterations considerably

1.4556965531732448e-10
47


#### Compare the calculation without preconditioner 

In [8]:
x, numIter = cg(A,b, rtol = 1e-12)
print(np.linalg.norm(A.dot(x) - b))
print(numIter)

6.186981003510558e-10
3689


The preconditioned matrix has much better distribution of eigenvalues, leading the a more stable convergence

# Your Task

Above, I solve the equation $D D^{\dagger} x = b$, but the 2003 paper solves $D x = b$. Therefore, the matrix you need to invert is below

## Summary I wrote at the beginning of my master thesis:

Dirac Lagrangian on the lattice using Wilson fermions:

$\overline{\psi}_x D_{xy}[U] \psi_y = \overline{\psi}_x \left[ (m+2) \delta_{xy} -\frac{1}{2}\sum_{\mu} \left(\Gamma_{+\mu} U_\mu(x) \delta_{x+\hat\mu,y} + \Gamma_{-\mu} U^\dagger_\mu(x-\hat\mu) \delta_{x-\hat\mu,y} \right) \right] \psi_y$,

Here $x$ is the vector (t,x)

In [30]:
m = -0.066
A = mg.D_slash # This is D already built by me
lattice_system = mg.calc # This contains all the information about the lattice than you need, but I am not sure if it's useful for you. My code is messy

where $\hat\mu$ is a unit vector pointing in direction $\mu$ ( $\mu$ = 0 is time direction and $\mu$ = 1 is spatial direction), $U_\mu(x)$ is the gauge link at site $x$ pointing at direction $\mu$

In [31]:
gauge_links = gauge_links.reshape(Nt,Nx,nDim, Nc,Nc) # I generated this myself, and this is only one of them

where $\Gamma_{\pm \mu} = \mathbb{1} \mp \gamma_\mu$, $\gamma_\mu$ are the Dirac $\gamma$-matrices. Note that the $\Gamma$'s are $2 \times 2$ matrices in 2 spacetime dimensions. Dirac and colour indices have been suppressed in the above formula for clarity. In our $2$-dimensional example, the $\gamma$-matrices are of size $2 \times 2$.
One possible choice for them is $\gamma_0=\sigma_3$ and $\gamma_1=\sigma_1$.

In [32]:
# Pauli matrices
Pauli = []
Pauli.append(np.array([[0, 1], [1, 0]]))
Pauli.append(np.array([[0,-1j], [1j, 0]]))
Pauli.append(np.array([[1, 0], [0, -1]]))

# Calculate Gamma above for different mu
Id = np.identity(2)
offdiagonal_spinor_x_plus = -0.5*(Id - Pauli[0])  
offdiagonal_spinor_x_minus = -0.5*(Id + Pauli[0])
offdiagonal_spinor_t_plus = -0.5*(Id - Pauli[2])
offdiagonal_spinor_t_minus = -0.5*(Id + Pauli[2])

## Things that can be helpful to understand

#### Gauge links

They live between two lattice points. 
Indices:
1. the position in the time of the starting lattice point
2. the position in the space of the starting lattice point
3. the direction to which it points, 0 is time direction, and 1 is space direction. 

4. and 5. the color indices, in this case, you only have one color


Let's look at specfic point on the lattice. The following is a gauge link which starts from the lattice point 12,12 and points in the positive time direction

In [33]:
nt = 12
nx = 12
mu = 0


gauge_links[nt,nx, mu]  


array([[0.661764+0.749713j]])

The following is a gauge link which starts from the lattice point 6,8 and points in the positive space direction

In [34]:
nt = 6
nx = 8
mu = 1
gauge_links[nt,nx, mu] 

array([[-0.138403+0.990376j]])

#### One important property to know:

$U_{-\mu}(x) = U^\dagger_\mu(x-\hat\mu)$

The following is a gauge link which starts from the lattice point 11,15 and points in the negative temporal direction

In [35]:
nt = 11
nx = 15
mu = 0
np.conjugate(gauge_links[nt - 1,nx, mu]) # Note the minus in the temporal direction

array([[0.974282+0.225332j]])

The following is a gauge link which starts from the lattice point 12,23 and points in the negative spatial direction

In [36]:
nt = 12
nx = 23
mu = 1
np.conjugate(gauge_links[nt,nx - 1, mu]) # Note the minus in the spatial direction

array([[0.862493-0.506068j]])

## Precalculated entries of the Dirac Matrix (from my code)

$\Gamma_{+\mu} U_\mu(x)$ 

In [37]:
offdiag_t_plus = lattice_system.offdiag_t_plus.reshape(Nt,Nx,Ns,Ns)
offdiag_x_plus = lattice_system.offdiag_x_plus.reshape(Nt,Nx,Ns,Ns)

In [38]:
offdiag_t_plus.shape

(128, 128, 2, 2)

(Nt,Nx,Ns,Ns)

#### Example 1:

In [39]:
nt = 12
nx = 12
mu = 0 

offdiag_t_plus[nt,nx]

array([[-0.      +0.j      , -0.      +0.j      ],
       [-0.      +0.j      , -0.661764-0.749713j]])

So what you see above is $\Gamma_{+t} U_t(t = 12, x = 12)$. Below you explicitly see this

In [40]:
# Gamma acts on the spinor space and the gauge link on the color space, therefore you need to kron it. Here, Nc = 1, so it's basically a multiplication of Gamma and U'
np.kron(offdiagonal_spinor_t_plus,gauge_links[nt,nx,mu,0,0]) 

array([[-0.      +0.j      , -0.      +0.j      ],
       [-0.      +0.j      , -0.661764-0.749713j]])

#### Example 2:

In [41]:
nt = 12
nx = 12
mu = 1
offdiag_x_plus[nt,nx]

array([[ 0.3770225+0.3284115j, -0.3770225-0.3284115j],
       [-0.3770225-0.3284115j,  0.3770225+0.3284115j]])

So what you see above is $\Gamma_{+x} U_x(t = 12, x = 12)$. Below you explicitly see this

In [42]:
np.kron(offdiagonal_spinor_x_plus,gauge_links[nt,nx, 1 ,0,0])

array([[ 0.3770225+0.3284115j, -0.3770225-0.3284115j],
       [-0.3770225-0.3284115j,  0.3770225+0.3284115j]])

$\Gamma_{-\mu} U^\dagger_\mu(x-\hat\mu)$

In [43]:
offdiag_t_minus = lattice_system.offdiag_t_minus.reshape(Nt,Nx,Ns,Ns)
offdiag_x_minus = lattice_system.offdiag_x_minus.reshape(Nt,Nx,Ns,Ns)

#### Example 3:

In [44]:
nt = 12
nx = 12
mu = 0 

offdiag_t_minus[nt,nx]

array([[-0.98083+0.194867j,  0.     +0.j      ],
       [ 0.     +0.j      ,  0.     +0.j      ]])

So what you see above is $\Gamma_{-t} U^\dagger_{-t}( t = 12-1, x = 12)$. Below you explicitly see this

In [45]:
np.kron(offdiagonal_spinor_t_minus,np.conjugate(gauge_links[nt - 1,nx,mu,0,0]))

array([[-0.98083+0.194867j,  0.     +0.j      ],
       [ 0.     +0.j      ,  0.     +0.j      ]])

#### Example 4:

In [46]:
nt = 12
nx = 12
mu = 1


offdiag_x_minus[nt,nx]

array([[0.151359+0.47654j, 0.151359+0.47654j],
       [0.151359+0.47654j, 0.151359+0.47654j]])

So what you see above is $\Gamma_{-x} U^\dagger_{-x}( t = 12, x = 12 - 1)$. Below you explicitly see this

In [47]:
np.kron(offdiagonal_spinor_x_minus,np.conjugate(gauge_links[nt,nx - 1, mu ,0,0]))

array([[0.151359+0.47654j, 0.151359+0.47654j],
       [0.151359+0.47654j, 0.151359+0.47654j]])

## Task 1:

Rebuild the original dirac matrix using the objects given above without looking at my code.

You can choose to use or not use my precomputed objects


(Do not forget antiperiodic boundaries in the temporal direction. This applies to the fermions)

In [48]:
np.random.seed(0)
b = np.random.randn(Nt*Nx*Ns) + 1j*np.random.randn(Nt*Nx*Ns) # A random vector as the pseudofermion field

In [49]:
A.dot(b)

array([ 3.42156492-2.99133457j,  4.77469303-3.42100349j,
       -2.38651842+1.14091977j, ...,  2.20167098-1.43554759j,
        1.51971237+1.19184527j, -3.64688596-1.06124401j])

In [50]:
# ----------- INSERT YOUR CODE HERE -------------------------------------------------
# Build the dirac matrix  D or a function which applies the dirac matrix to a pseudofermion field
# It should give you the same result as above
# np.linalg.norm(D.dot(b) - A.dot(b)) should be around 0

